# 이전 대화를 기억하는 Chain 생성방법

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 이전 대화내용을 기억하는 multi-turn Chain

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# llm 생성
llm = ChatOpenAI(model_name="gpt-4o")

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

대화를 기록하는 체인 생성(`chain_with_history`)

In [3]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

In [4]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

첫 번째 질문 실행

In [ ]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "나의 이름은 테디입니다."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'안녕하세요, 테디! 만나서 반갑습니다. 도움이 필요하신 것이 있으면 말씀해주세요.'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Bad request: invalid \'dotted_order\': dotted_order 7a6898bd-d7aa-4f3b-905f-cb81781fdc29 has timestamp 2025-02-13 07:00:25.323646 +0000 UTC earlier than parent timestamp 2025-02-13 07:00:29.471946 +0000 UTC for run_id:7a6898bd-d7aa-4f3b-905f-cb81781fdc29 trace_id:2641fa19-95d3-4c26-8543-3983f741eb29 dotted_order:20250213T070029461017Z2641fa19-95d3-4c26-8543-3983f741eb29.20250213T070029470531Z74f69630-eb7a-40ef-8b0c-1478e2182fc4.20250213T070029471946Z39e3a44d-8f64-418c-98e9-24f49a8be41e.20250213T070025323646Z7a6898bd-d7aa-4f3b-905f-cb81781fdc29 parent_run_id:39e3a44d-8f64-418c-98e9-24f49a8be41e"}\n')


이어서 질문 실행

In [6]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'당신의 이름은 테디입니다.'

아래는 `session_id`가 다른 경우 새로운 세션이 생성되는 경우입니다.

In [7]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc1234"}},
)

[대화 세션ID]: abc1234


'죄송하지만 당신의 이름을 알 수 있는 방법이 없습니다.'